In [68]:
import os
from random import shuffle
from datasets import load_dataset, load_metric
from transformers import AutoConfig, RobertaTokenizer, RobertaForSequenceClassification, AdapterConfig, AdapterArguments
from transformers import AutoModelForSequenceClassification
from transformers import RobertaConfig, RobertaModelWithHeads, default_data_collator, AutoTokenizer, AutoModelWithHeads
import numpy as np
from transformers import TrainingArguments, Trainer, EvalPrediction, DataCollatorWithPadding, HfArgumentParser

os.environ["WANDB_DISABLED"] = "true"
# os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [36]:
datasets = load_dataset("glue", "mnli")
num_labels = 3

Reusing dataset glue (/home/nlp/.cache/huggingface/datasets/glue/mnli/1.0.0/7c99657241149a24692c402a5c3f34d4c9f1df5ac2e4c3759fadea38f6cb29c4)


In [73]:
label_list = datasets["train"].features["label"].names

In [80]:
adapter_args = AdapterArguments

In [75]:
config = AutoConfig.from_pretrained(
        "roberta-large",
        num_labels=num_labels,
        finetuning_task="mnli",
#         cache_dir=model_args.cache_dir,
    )
tokenizer = AutoTokenizer.from_pretrained(
    "roberta-large"
#     cache_dir=model_args.cache_dir,
#     use_fast=model_args.use_fast_tokenizer,
)
# We use the AutoModelWithHeads class here for better adapter support.
model = AutoModelWithHeads.from_pretrained(
#     model_args.model_name_or_path,
    "roberta-large",
#     from_tf=bool(".ckpt" in model_args.model_name_or_path),
    config=config,
#     cache_dir=model_args.cache_dir,
)
model.add_classification_head(
    "mnli",
    num_labels=num_labels,
    id2label={i: v for i, v in enumerate(label_list)} if num_labels > 0 else None,
)

loading configuration file https://huggingface.co/roberta-large/resolve/main/config.json from cache at /home/nlp/.cache/huggingface/transformers/dea67b44b38d504f2523f3ddb6acb601b23d67bee52c942da336fa1283100990.94cae8b3a8dbab1d59b9d4827f7ce79e73124efa6bb970412cd503383a95f373
Model config RobertaConfig {
  "adapters": {
    "adapters": {},
    "config_map": {}
  },
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "eos_token_id": 2,
  "finetuning_task": "mnli",
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 1024,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2
  },
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 16,
  "num_hidden_layers": 24,
  "pa

In [83]:
adapter_args = AdapterArguments(train_adapter=True, 
                                load_adapter = "/home/nlp/experiments/adapter_less_data/mnli")
adapter_config = AdapterConfig.load(
                adapter_args.adapter_config,
                non_linearity=adapter_args.adapter_non_linearity,
                reduction_factor=adapter_args.adapter_reduction_factor,
            )

In [93]:
model.load_adapter(
                adapter_args.load_adapter,
                config=adapter_config,
                load_as="mnli",
            );
model.train_adapter(["mnli"])
    # Set the adapters to be used in every forward pass
#     if lang_adapter_name:
#         model.set_active_adapters(ac.Stack(lang_adapter_name, task_name))
#     else:
model.set_active_adapters("mnli")

Loading module configuration from /home/nlp/experiments/adapter_less_data/mnli/adapter_config.json
Overwriting existing adapter 'mnli'.
Loading module weights from /home/nlp/experiments/adapter_less_data/mnli/pytorch_adapter.bin
Loading module configuration from /home/nlp/experiments/adapter_less_data/mnli/head_config.json
Overwriting existing head 'mnli'
Adding head 'mnli' with config {'head_type': 'classification', 'num_labels': 3, 'layers': 2, 'activation_function': 'tanh', 'label2id': {'contradiction': 2, 'entailment': 0, 'neutral': 1}, 'use_pooler': False, 'bias': True}.
Loading module weights from /home/nlp/experiments/adapter_less_data/mnli/pytorch_model_head.bin


In [88]:
label_to_id = {v: i for i, v in enumerate(label_list)}

In [ ]:
 if adapter_args.train_adapter:
    task_name = data_args.task_name or "glue"
    # check if adapter already exists, otherwise add it
    if task_name not in model.config.adapters:
        # resolve the adapter config
        
        # otherwise, add a fresh adapter
#         else:
#             model.add_adapter(task_name, config=adapter_config)
    # optionally load a pre-trained language adapter
#     if adapter_args.load_lang_adapter:
#         # resolve the language adapter config
#         lang_adapter_config = AdapterConfig.load(
#             adapter_args.lang_adapter_config,
#             non_linearity=adapter_args.lang_adapter_non_linearity,
#             reduction_factor=adapter_args.lang_adapter_reduction_factor,
#         )
#         # load the language adapter from Hub
#         lang_adapter_name = model.load_adapter(
#             adapter_args.load_lang_adapter,
#             config=lang_adapter_config,
#             load_as=adapter_args.language,
#         )
#     else:
#         lang_adapter_name = None
#     # Freeze all model weights except of those of this adapter


In [56]:
tokenizer = RobertaTokenizer.from_pretrained("roberta-base")
# model = AutoModelForSequenceClassification.from_pretrained("/home/nlp/experiments/adapter_less_data/")

train_dataset = load_dataset("multi_nli", split="train")
valid_dataset = load_dataset("multi_nli", split="validation_matched")

loading file https://huggingface.co/roberta-base/resolve/main/vocab.json from cache at /home/nlp/.cache/huggingface/transformers/d3ccdbfeb9aaa747ef20432d4976c32ee3fa69663b379deb253ccfce2bb1fdc5.d67d6b367eb24ab43b08ad55e014cf254076934f71d832bbab9ad35644a375ab
loading file https://huggingface.co/roberta-base/resolve/main/merges.txt from cache at /home/nlp/.cache/huggingface/transformers/cafdecc90fcab17011e12ac813dd574b4b3fea39da6dd817813efa010262ff3f.5d12962c5ee615a4c803841266e9c3be9a691a924f72d395d3a6c6c81157788b
loading file https://huggingface.co/roberta-base/resolve/main/added_tokens.json from cache at None
loading file https://huggingface.co/roberta-base/resolve/main/special_tokens_map.json from cache at None
loading file https://huggingface.co/roberta-base/resolve/main/tokenizer_config.json from cache at None
loading file https://huggingface.co/roberta-base/resolve/main/tokenizer.json from cache at /home/nlp/.cache/huggingface/transformers/d53fc0fa09b8342651efd4073d75e19617b3e51287

In [57]:


train_dataset = train_dataset.map(preprocess_function, batched=False)
valid_dataset = valid_dataset.map(preprocess_function, batched=False)
# The transformers model expects the target class column to be named "labels"
train_dataset.rename_column_("label", "labels")
valid_dataset.rename_column_("label", "labels")
# Transform to pytorch tensors and only output the required columns
train_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])
valid_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

Loading cached processed dataset at /home/nlp/.cache/huggingface/datasets/multi_nli/plain_text/1.0.0/9969e1448f410fe7c6c688a84bfcb61312d0a3f2741d57341c26ef99f28a5451/cache-2f48bf8df98a5fe9.arrow
Loading cached processed dataset at /home/nlp/.cache/huggingface/datasets/multi_nli/plain_text/1.0.0/9969e1448f410fe7c6c688a84bfcb61312d0a3f2741d57341c26ef99f28a5451/cache-f28fef05c2344eb8.arrow


In [58]:
random_sequence = [i for i in range(39200)][:1024]
shuffle(random_sequence)
train_dataset = train_dataset.select(random_sequence)

In [59]:
config = RobertaConfig.from_pretrained(
    "roberta-large",
    num_labels=3,
    finetuning_task='mnli',
)
model = RobertaModelWithHeads.from_pretrained(
    "roberta-large",
    config=config,
)

loading configuration file https://huggingface.co/roberta-large/resolve/main/config.json from cache at /home/nlp/.cache/huggingface/transformers/dea67b44b38d504f2523f3ddb6acb601b23d67bee52c942da336fa1283100990.94cae8b3a8dbab1d59b9d4827f7ce79e73124efa6bb970412cd503383a95f373
Model config RobertaConfig {
  "adapters": {
    "adapters": {},
    "config_map": {}
  },
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "eos_token_id": 2,
  "finetuning_task": "mnli",
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 1024,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2
  },
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 16,
  "num_hidden_layers": 24,
  "pa

In [60]:
adapter_args = AdapterArguments(train_adapter=True, 
                                load_adapter = "/home/nlp/experiments/adapter_less_data/mnli",
                        adapter_non_linearity="relu", adapter_reduction_factor=16)
adapter_config = AdapterConfig.load(
                adapter_args.adapter_config,
                non_linearity=adapter_args.adapter_non_linearity,
                reduction_factor=adapter_args.adapter_reduction_factor,
            )

In [61]:
adapter_name = model.load_adapter(adapter_args.load_adapter, config=adapter_config,
                                 load_as="mnli")
model.train_adapter("mnli")
model.set_active_adapters(adapter_name)

Loading module configuration from /home/nlp/experiments/adapter_less_data/mnli/adapter_config.json
Adding adapter 'mnli'.
Loading module weights from /home/nlp/experiments/adapter_less_data/mnli/pytorch_adapter.bin
Loading module configuration from /home/nlp/experiments/adapter_less_data/mnli/head_config.json
Adding head 'mnli' with config {'head_type': 'classification', 'num_labels': 3, 'layers': 2, 'activation_function': 'tanh', 'label2id': {'contradiction': 2, 'entailment': 0, 'neutral': 1}, 'use_pooler': False, 'bias': True}.
Loading module weights from /home/nlp/experiments/adapter_less_data/mnli/pytorch_model_head.bin


In [62]:
# model.add_adapter("mnli", config=adapter_config)
# model.add_classification_head(
#     "mnli",
#     num_labels=3,
#     id2label={ 0: "0", 1: "1", 2: "2"}
#   )
# # Activate the adapter
# model.train_adapter("mnli")
# model.set_active_adapters("mnli")

In [63]:
# data_collator = DataCollatorWithPadding(tokenizer, pad_to_multiple_of=8)
metric = load_metric("glue", "mnli")

In [64]:
model.eval();

In [65]:
training_args = TrainingArguments(
    learning_rate=1e-4,
    num_train_epochs=12,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=256,
    evaluation_strategy = 'epoch',
#     logging_steps=10,
#     n_gpu=1,
    output_dir="~/experiments/adapter_less_data",
    overwrite_output_dir=False,
    # The next line is important to ensure the dataset labels are properly passed to the model
    remove_unused_columns=False,
)

def compute_metrics(p: EvalPrediction):
    preds = p.predictions[0] if isinstance(p.predictions, tuple) else p.predictions
    preds = np.argmax(preds, axis=1)
#     if data_args.task_name is not None:
    result = metric.compute(predictions=preds, references=p.label_ids)
    if len(result) > 1:
        result["combined_score"] = np.mean(list(result.values())).item()
    return result

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
Using the `WAND_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [66]:
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=default_data_collator,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    do_save_full_model=False,
    do_save_adapters=True,
)

In [67]:
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 9815
  Batch size = 256


{'eval_loss': 3.1259284019470215,
 'eval_accuracy': 0.5602649006622517,
 'eval_runtime': 65.9337,
 'eval_samples_per_second': 148.862,
 'eval_steps_per_second': 0.592}

In [25]:
hans_data = load_dataset("hans", split="validation")

Reusing dataset hans (/home/nlp/.cache/huggingface/datasets/hans/plain_text/1.0.0/ccd7261ed79f0e126a724d1c0b6d94d8e029b9ec309c8e6c5dbece3538ab7f3b)
